Code by Marvin Kinz, m.kinz@stud.uni-heidelberg.de
# Downsampling of objects

## Create Downsampling data
Creates files, which can be used to downsample the point clouds of objects and slices. CleanUpUE4Data has to be run once before. For the supplied dataset this has already been done.


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
import glob
from pathlib import Path 

def getGreedyPerm(D):
    """
    A Naive O(N^2) algorithm to do furthest points sampling based on (https://gist.github.com/ctralie/128cc07da67f1d2e10ea470ee2d23fe8)
    
    Parameters
    ----------
    D : ndarray (N, N) 
        An NxN distance matrix for points
    Return
    ------
    tuple (list, list) 
        permutation (N-length array of indices)
    """
    
    N = D.shape[0]
    #By default, takes the first point in the list to be the
    #first point in the permutation, but could be random
    perm = np.zeros(N, dtype=np.int64)
    ds = D[0, :]
    for i in range(1, N):
        idx = np.argmax(ds)
        perm[i] = idx
        ds = np.minimum(ds, D[idx, :])
    return perm

def Downsample_File(obj):
    """
    Calculates sampling order for one object

    params
    ------
    obj : string
        Object to create downsampling order for

    returns
    -------
    Stores sampling order for point clouds as .txt files, which can be loaded to easily sample afterwards.
    """

    X=np.loadtxt(obj)
    D = pairwise_distances(X, metric='euclidean')
    perm= getGreedyPerm(D)
    np.savetxt(f"{obj[:-4]}_SampleOrder.txt",perm)

def Downsample(ShapeFolder):
    """
    Calculates sampling order for all objects for folder in format of "SimulationResults". This has already been done in the supplied dataset.

    params
    ------
    ShapeFolder : string
        Folder with folders of simulation outputs of multiple runs inside

    returns
    -------
    Stores sampling order for all point clouds as .txt files, which can be loaded to easily sample afterwards.
    
    """
    shapes=glob.glob(f"{ShapeFolder}/*")
    for shape in shapes:
        objects=glob.glob(f"{shape}/Initial/*.xyz")
        for obj in objects:
            Downsample_File(obj)
        objects=glob.glob(f"{shape}//*.xyz")
        gravities=glob.glob(f"{shape}/G*")
        for grav in gravities:
            objects=glob.glob(f"{grav}/Slices/*.xyz")
            for obj in objects:
                Downsample_File(obj)

In [ ]:
Downsample("2 Deformation and Slicing/SimulationResults")

## Downsample
Uses files created in the function before to create the actual downsampled files.

In [ ]:
import numpy as np
import os
import glob
from pathlib import Path 
import pandas as pd

def matched_indices(x, y): #Based on https://stackoverflow.com/a/71458357
    # Indices to sort y
    y_argsort = y.argsort()

    # Indices in sorted y of corresponding x elements, flat
    x_in_y_sort = y.searchsorted(x, sorter=y_argsort)

    # Indices in y of corresponding x elements, append -1 for elements put at the end, which do not exist
    x_in_y = np.concatenate((y_argsort,[-1]))[x_in_y_sort]

    # Check for inequality at each y index to mask invalid indices
    mask = np.isin(x,y[x_in_y])
    return mask, x_in_y[mask]

def CreateSamples_File(obj,folder_order,folder_out,N):
    """
    Creates one sampled file.
    
    params
    ------
    obj : string
        Object to sample
    folder_order : string
        Folder, where the correct sampling order is stored in
    folder_out : string
        Folder, where the sampled object should be saved
    N : int (0,inf)
        Number of sampled points

    returns
    -------
    Stores sampled point cloud as .xyz
    """
    X=np.loadtxt(obj)
    if len(X) < N: N=len(X)
    ord=np.loadtxt(f"{folder_order}/{Path(obj).stem}_SampleOrder.txt",dtype=int)
    np.savetxt(f"{folder_out}/{Path(obj).stem}.xyz",X[ord[:N]])
    
def CreateSamples_Views(folder_in,folder_out,folder_order,N):
    """
    Creates sampled view files
    
    params
    ------
    folder_in: string
        Folder, where the View folder is placed in
    folder_out : string
        Folder, where the sampled object should be saved
    folder_order : string
        Folder, where the correct sampling order is stored in
    N : int (0,inf)
        Number of sampled points

    returns
    -------
    Stores sampled point clouds as .xyz together with correspondence
    """
    if os.path.exists(f"{folder_in}/Views"):  
        os.makedirs(f"{folder_out}/Views", exist_ok=True)          
        df = pd.read_csv (f'{folder_in}/Views/Properties.csv')
        T=df.Target
        for i,obj in enumerate(df.Filename):
            X=np.loadtxt(obj)
            name=Path(obj).stem
            ind=np.loadtxt(f"{folder_in}/Views/{name}_Correspondence.txt")
            ord=np.loadtxt(f"{folder_order}/{T[i]}_SampleOrder.txt",dtype=int)
            mask,cor=matched_indices(ind,ord[:N])
            np.savetxt(f"{folder_out}/Views/{name}.xyz",X[mask])
            np.savetxt(f"{folder_out}/Views/{name}_Correspondence.txt",ord[cor])

def CreateSamples(ShapeFolder, N):
    """
    Creates sampled point clouds with specified number of Points in new main folder "{ShapeFolder}_SampleSize{N}" with structure of the old by sampling the originals using the before calculated sample order.

    params
    ------
    ShapeFolder : string
        Folder with folders of simulation outputs of multiple runs inside
    N : int (0,inf)
        Number of sampled points

    returns
    -------
    Stores sampled point clouds as .xyz in new ShapeFolder with sample number appended.
    """ 
    
    
    FolderOut=f"{ShapeFolder}_SampleSize{N}"
    os.makedirs(FolderOut, exist_ok=True)
    shapes=glob.glob(f"{ShapeFolder}/*")
    for shape in shapes:
        FO1=f"{FolderOut}/{Path(shape).stem}"
                
        gravities=glob.glob(f"{shape}/G*")
        for grav in gravities:
            FO2=f"{FO1}/{Path(grav).stem}/Deformed"
            os.makedirs(FO2, exist_ok=True)
            F=f"{grav}/Deformed"
            folder_order=f"{shape}/Initial"
            objects=glob.glob(f"{F}/*.xyz")
            for obj in objects:
                CreateSamples_File(obj,folder_order,FO2,N)
            CreateSamples_Views(F,FO2,folder_order,N)

            FO2=f"{FO1}/{Path(grav).stem}/Slices"
            os.makedirs(FO2, exist_ok=True)
            F=f"{grav}/Slices"
            objects=glob.glob(f"{F}/*.xyz")
            for obj in objects:
                CreateSamples_File(obj,F,FO2,N)
            CreateSamples_Views(F,FO2,F,N)

            FO2=f"{FO1}/{Path(grav).stem}/Slices deformed"
            os.makedirs(FO2, exist_ok=True)
            F=f"{grav}/Slices deformed"
            folder_order=f"{grav}/Slices"
            objects=glob.glob(f"{F}/*.xyz")
            for obj in objects:
                CreateSamples_File(obj,folder_order,FO2,N)
            CreateSamples_Views(F,FO2,folder_order,N)

        FO2=f"{FO1}/Initial"
        os.makedirs(FO2, exist_ok=True)
        F=f"{shape}/Initial"
        objects=glob.glob(f"{F}/*.xyz")
        for obj in objects:
            CreateSamples_File(obj,F,FO2,N)
        CreateSamples_Views(F,FO2,F,N)

In [ ]:
#CHANGE NUMBER IN THIS CALL FOR DIFFERENT SAMPLE SIZES
CreateSamples("2 Deformation and Slicing/SimulationResults",1500)